In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
!pip install ipywidgets

In [3]:
!pip install import-ipynb

In [4]:
import import_ipynb

In [5]:
import utils, get_loader_v3, model

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from get_loader_v3.ipynb
     |████████████████████████████████| 13.9 MB 1.8 MB/s            


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0-py3-none-any.whl (13.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
           Image-ID                                               Head
395  Bimaculatus_46  Head=False, Anterior=Y, IASpace=B, Scutellum=Y...
396  Bimaculatus_47  Head=Y, Anterior=Y, IASpace=B, Scutellum=Y, T1...
397  Bimaculatus_48  Head=B, Anterior=Y, IASpace=B, Scutellum=Y, T1...
398  Bimaculatus_49  Head=B, Anterior=Y, IASpace=B, Scutellum=Y, T1...
399  Bimaculatus_50  Head=B, Anterior=Y, IASpace=B, Scutellum=Y, T1...
torch.Size([32, 3, 224, 224])
torch.Size([51, 32])
torch.Size([32, 3, 224, 224])
torch.Size([53, 32])
to

In [6]:
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from utils import save_checkpoint, load_checkpoint, print_examples
from get_loader_v3 import get_loader
from model import CNNtoRNN

In [7]:
def train():
    transform = transforms.Compose(
        [
            transforms.Resize((356, 356)),
            transforms.RandomCrop((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

    train_loader, dataset = get_loader(
        root_folder="../../Bombus",
        annotation_file="../Beemachine_Experiments_Logs.xlsx",
        transform=transform,
        num_workers=2,
    )

    torch.backends.cudnn.benchmark = True
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    load_model = False
    save_model = False
    train_CNN = False

    # Hyperparameters
    embed_size = 256
    hidden_size = 256
    vocab_size = len(dataset.vocab)
    num_layers = 1
    learning_rate = 3e-4
    num_epochs = 100

    # for tensorboard
    writer = SummaryWriter("runs/bumble-bees")
    step = 0

    # initialize model, loss etc
    model = CNNtoRNN(embed_size, hidden_size, vocab_size, num_layers).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Only finetune the CNN
    for name, param in model.encoderCNN.inception.named_parameters():
        if "fc.weight" in name or "fc.bias" in name:
            param.requires_grad = True
        else:
            param.requires_grad = train_CNN

    if load_model:
        step = load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)

    model.train()

    for epoch in range(num_epochs):
        # Uncomment the line below to see a couple of test cases
        print_examples(model, device, dataset)

        if save_model:
            checkpoint = {
                "state_dict": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "step": step,
            }
            save_checkpoint(checkpoint)

        for idx, (imgs, captions) in tqdm(
            enumerate(train_loader), total=len(train_loader), leave=False
        ):
            imgs = imgs.to(device)
            captions = captions.to(device)

            outputs = model(imgs, captions[:-1])
            loss = criterion(
                outputs.reshape(-1, outputs.shape[2]), captions.reshape(-1)
            )

            writer.add_scalar("Training loss", loss.item(), global_step=step)
            step += 1

            optimizer.zero_grad()
            loss.backward(loss)
            optimizer.step()



In [8]:
if __name__ == "__main__":
    train()

           Image-ID                                               Head
395  Bimaculatus_46  Head=False, Anterior=Y, IASpace=B, Scutellum=Y...
396  Bimaculatus_47  Head=Y, Anterior=Y, IASpace=B, Scutellum=Y, T1...
397  Bimaculatus_48  Head=B, Anterior=Y, IASpace=B, Scutellum=Y, T1...
398  Bimaculatus_49  Head=B, Anterior=Y, IASpace=B, Scutellum=Y, T1...
399  Bimaculatus_50  Head=B, Anterior=Y, IASpace=B, Scutellum=Y, T1...
Example 1 Ground Truth: Head = B, Anterior = Y, IASpace = B, Scutellum = Y, T1 = Y, T2 = Y / O, T3 = B, T4 = B, T5 = B, T6 = B, Corbicular = false
Example 1 Model Prediction: / b iaspace false false , , scutellum scutellum scutellum corbicular <PAD> <SOS> / b t1 / b iaspace false false , , scutellum scutellum scutellum corbicular <PAD> <SOS> / b t1 / b iaspace false false , , scutellum scutellum scutellum corbicular <PAD> <SOS> / b t1 / b
reference: [['<sos> head = b', 'anterior = y', 'iaspace = b', 'scutellum = y', 't1 = y', 't2 = y / o', 't3 = b', 't4 = b', 't5 = b'

/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Example 1 Ground Truth: Head = B, Anterior = Y, IASpace = B, Scutellum = Y, T1 = Y, T2 = Y / O, T3 = B, T4 = B, T5 = B, T6 = B, Corbicular = false
Example 1 Model Prediction: <SOS> head = b , = b , = b , = b , = b , = b , = b , = b , = b , = b , = b , = b , = b , = b , = b , = b ,
reference: [['<sos> head = b', 'anterior = y', 'iaspace = b', 'scutellum = y', 't1 = y', 't2 = y / o', 't3 = b', 't4 = b', 't5 = b', 't6 = b', 'corbicular = false <eos>']]
c2: ['<sos> head = b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '']
BLEU 1-gram score: 0.0588235294117647
BLEU 2-gram score: 3.6178266619513134e-155
BLEU 3-gram score: 3.493004785859614e-204
BLEU 4-gram score: 8.972141065609098e-232


/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Example 1 Ground Truth: Head = B, Anterior = Y, IASpace = B, Scutellum = Y, T1 = Y, T2 = Y / O, T3 = B, T4 = B, T5 = B, T6 = B, Corbicular = false
Example 1 Model Prediction: <SOS> head = b , = b , = b , = b , = b , = b , = b , = b , = b , = b , = b , = b , = b , = b , = b , = b ,
reference: [['<sos> head = b', 'anterior = y', 'iaspace = b', 'scutellum = y', 't1 = y', 't2 = y / o', 't3 = b', 't4 = b', 't5 = b', 't6 = b', 'corbicular = false <eos>']]
c2: ['<sos> head = b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '= b', '']
BLEU 1-gram score: 0.0588235294117647
BLEU 2-gram score: 3.6178266619513134e-155
BLEU 3-gram score: 3.493004785859614e-204
BLEU 4-gram score: 8.972141065609098e-232


...

Example 1 Ground Truth: Head = B, Anterior = Y, IASpace = B, Scutellum = Y, T1 = Y, T2 = Y / O, T3 = B, T4 = B, T5 = B, T6 = B, Corbicular = false
Example 1 Model Prediction: <SOS> head = b , anterior = y , iaspace = b , scutellum = y , t1 = y , t2 = y / o , t3 = b , t4 = b , t5 = b , t6 = b , corbicular = false <EOS>
reference: [['<sos> head = b', 'anterior = y', 'iaspace = b', 'scutellum = y', 't1 = y', 't2 = y / o', 't3 = b', 't4 = b', 't5 = b', 't6 = b', 'corbicular = false <eos>']]
c2: ['<sos> head = b', 'anterior = y', 'iaspace = b', 'scutellum = y', 't1 = y', 't2 = y / o', 't3 = b', 't4 = b', 't5 = b', 't6 = b', 'corbicular = false <eos>']
BLEU 1-gram score: 1.0
BLEU 2-gram score: 1.0
BLEU 3-gram score: 1.0
BLEU 4-gram score: 1.0


Example 1 Ground Truth: Head = B, Anterior = Y, IASpace = B, Scutellum = Y, T1 = Y, T2 = Y / O, T3 = B, T4 = B, T5 = B, T6 = B, Corbicular = false
Example 1 Model Prediction: <SOS> head = b , anterior = y , iaspace = b , scutellum = y , t1 = y , t2 = y / o , t3 = b , t4 = b , t5 = b , t6 = b , corbicular = false <EOS>
reference: [['<sos> head = b', 'anterior = y', 'iaspace = b', 'scutellum = y', 't1 = y', 't2 = y / o', 't3 = b', 't4 = b', 't5 = b', 't6 = b', 'corbicular = false <eos>']]
c2: ['<sos> head = b', 'anterior = y', 'iaspace = b', 'scutellum = y', 't1 = y', 't2 = y / o', 't3 = b', 't4 = b', 't5 = b', 't6 = b', 'corbicular = false <eos>']
BLEU 1-gram score: 1.0
BLEU 2-gram score: 1.0
BLEU 3-gram score: 1.0
BLEU 4-gram score: 1.0


Example 1 Ground Truth: Head = B, Anterior = Y, IASpace = B, Scutellum = Y, T1 = Y, T2 = Y / O, T3 = B, T4 = B, T5 = B, T6 = B, Corbicular = false
Example 1 Model Prediction: <SOS> head = b , anterior = y , iaspace = b , scutellum = y , t1 = y , t2 = y / o , t3 = b , t4 = b , t5 = b , t6 = b , corbicular = false <EOS>
reference: [['<sos> head = b', 'anterior = y', 'iaspace = b', 'scutellum = y', 't1 = y', 't2 = y / o', 't3 = b', 't4 = b', 't5 = b', 't6 = b', 'corbicular = false <eos>']]
c2: ['<sos> head = b', 'anterior = y', 'iaspace = b', 'scutellum = y', 't1 = y', 't2 = y / o', 't3 = b', 't4 = b', 't5 = b', 't6 = b', 'corbicular = false <eos>']
BLEU 1-gram score: 1.0
BLEU 2-gram score: 1.0
BLEU 3-gram score: 1.0
BLEU 4-gram score: 1.0


Example 1 Ground Truth: Head = B, Anterior = Y, IASpace = B, Scutellum = Y, T1 = Y, T2 = Y / O, T3 = B, T4 = B, T5 = B, T6 = B, Corbicular = false
Example 1 Model Prediction: <SOS> head = b , anterior = y , iaspace = b , scutellum = y , t1 = y , t2 = y / o , t3 = b , t4 = b , t5 = b , t6 = b , corbicular = false <EOS>
reference: [['<sos> head = b', 'anterior = y', 'iaspace = b', 'scutellum = y', 't1 = y', 't2 = y / o', 't3 = b', 't4 = b', 't5 = b', 't6 = b', 'corbicular = false <eos>']]
c2: ['<sos> head = b', 'anterior = y', 'iaspace = b', 'scutellum = y', 't1 = y', 't2 = y / o', 't3 = b', 't4 = b', 't5 = b', 't6 = b', 'corbicular = false <eos>']
BLEU 1-gram score: 1.0
BLEU 2-gram score: 1.0
BLEU 3-gram score: 1.0
BLEU 4-gram score: 1.0


Example 1 Ground Truth: Head = B, Anterior = Y, IASpace = B, Scutellum = Y, T1 = Y, T2 = Y / O, T3 = B, T4 = B, T5 = B, T6 = B, Corbicular = false
Example 1 Model Prediction: <SOS> head = b , anterior = y , iaspace = b , scutellum = y , t1 = y , t2 = y / o , t3 = b , t4 = b , t5 = b , t6 = b , corbicular = false <EOS>
reference: [['<sos> head = b', 'anterior = y', 'iaspace = b', 'scutellum = y', 't1 = y', 't2 = y / o', 't3 = b', 't4 = b', 't5 = b', 't6 = b', 'corbicular = false <eos>']]
c2: ['<sos> head = b', 'anterior = y', 'iaspace = b', 'scutellum = y', 't1 = y', 't2 = y / o', 't3 = b', 't4 = b', 't5 = b', 't6 = b', 'corbicular = false <eos>']
BLEU 1-gram score: 1.0
BLEU 2-gram score: 1.0
BLEU 3-gram score: 1.0
BLEU 4-gram score: 1.0
